In [1]:
# This makes a first contrast curve for Altair

# created 2018 Sept. 20 by E.S.

## SECTION TO INITIALIZE

In [1]:
# import stuff

import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition, \
                                    StackingAndSubsampling
#from PynPoint.ProcessingModules.StackingAndSubsampling import DerotateAndStackModule
from PynPoint.ProcessingModules.FluxAndPosition import FakePlanetModule
#from PynPoint.Util import AnalysisTools

In [2]:
# define workspaces and initialize Pypeline

stem = ('/home/../../media/unasemaje/Elements/lbti_data_reduction/'+
        '180507_fizeau_altair/06_winnowed/')

working_place = stem+"/junk_test_working_place/"
input_place = stem
output_place = stem+"/junk_test_output_place/"

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [3]:
## IF THE *INI FILE WAS NEWLY GENERATED, EDIT THE *INI FILE TO SET
## PIXSCALE = 0.0107

## SECTION TO READ IN DATA

In [4]:
# read in science FITS files 
# (should have PARANG=0 in all headers, so as to keep the PSF in the same original orientation on the array; 
# we need to wait to correct for PARANG later)

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [5]:
# read in PSF reference FITS files (i.e., unsaturated frames)
# (these do not have any PARANG in the header, but it probably doesn't matter)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place,
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

In [6]:
# make PCA basis set, calculate image residuals after subtracting the PCA reconstructions of the PSF

pca_pca_subt = PcaPsfSubtractionModule(pca_numbers=(100, ),
                                       name_in="pca",
                                       images_in_tag="science",
                                       reference_in_tag="ref_psf",
                                       res_mean_tag="mean_residuals",
                                       res_median_tag="median_residuals",
                                       res_arr_out_tag="all_resids",
                                       res_rot_mean_clip_tag="resid_rot",
                                       basis_out_tag="pca_components",
                                       subtract_mean=True)

pipeline.add_module(pca_pca_subt)

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [7]:
# write out PCA basis set

write_pca_basis = FitsWritingModule(file_name="test_pca_basis.fits",
                              name_in="write_pca_basis",
                              output_dir=output_place,
                              data_tag="pca_components")

pipeline.add_module(write_pca_basis)

In [8]:
# write out PCA residuals, all

write_pca_resids_all = FitsWritingModule(file_name="test_pca_resids_all.fits",
                              name_in="write_pca_resids_all",
                              output_dir=output_place,
                              data_tag="all_resids")

#pipeline.add_module(write_pca_resids_all)

In [9]:
# write out PCA residuals, median

write_pca_resids_median = FitsWritingModule(file_name="test_pca_resids_median.fits",
                              name_in="write_pca_resids_median",
                              output_dir=output_place,
                              data_tag="median_residuals")

pipeline.add_module(write_pca_resids_median)

In [10]:
# write out PCA residuals, mean

write_pca_resids_mean = FitsWritingModule(file_name="test_pca_resids_mean.fits",
                              name_in="write_pca_resids_mean",
                              output_dir=output_place,
                              data_tag="mean_residuals")

pipeline.add_module(write_pca_resids_mean)

In [11]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:167: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:197: UserWarning: Non-static attribute PUPIL (=ESO ADA PUPILPOS) not found in the FITS header.
  "FITS header." % (item, fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:197: UserWarning: Non-static attribute DEC (=DE

Running FitsReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Constructing PSF model... [DONE]
Creating residuals... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]
